In [1]:
#! uv pip install --extra-index-url=https://pypi.nvidia.com "cudf-cu12==24.12.*" "cuml-cu12==24.12.*"

In [2]:
import polars as pl
import numpy as np
import cuml

In [3]:
df = pl.read_parquet(
    "mtg_embeddings.parquet",
)


df

name,scryfallId,manaCost,type,text,power,toughness,loyalty,rarities,sets,embedding
str,str,str,str,str,str,str,str,list[enum],list[enum],"array[f32, 768]"
"""""Ach! Hans, Run!""""","""84f2c8f5-8e11-4639-b7de-00e4a2…","""{2}{R}{R}{G}{G}""","""Enchantment""","""At the beginning of your upkee…",null,null,null,"[""rare""]","[""UNH""]","[0.021458, -0.036102, … -0.00607]"
"""""Brims"" Barone, Midway Mobster""","""68832214-2943-4253-8884-ffa490…","""{3}{W}{B}""","""Legendary Creature — Human Rog…","""When ~ enters, put a +1/+1 cou…","""5""","""4""",null,"[""uncommon""]","[""UNF""]","[0.000261, 0.025207, … 0.005641]"
"""""Lifetime"" Pass Holder""","""42293306-aaea-4542-8df4-813823…","""{B}""","""Creature — Zombie Guest""","""This creature enters tapped.\n…","""2""","""1""",null,"[""rare""]","[""UNF""]","[-0.004467, -0.016707, … 0.001401]"
"""""Name Sticker"" Goblin""","""fd1442b4-da59-4042-835f-143c8d…","""{2}{R}""","""Creature — Goblin Guest""","""When this creature enters from…","""2""","""2""",null,"[""common""]","[""UNF""]","[-0.03243, -0.006905, … -0.026888]"
"""""Rumors of My Death . . .""""","""cb3587b9-e727-4f37-b4d6-1baa73…","""{2}{B}""","""Enchantment""","""{3}{B}, Exile a permanent you …",null,null,null,"[""uncommon""]","[""UST""]","[-0.008719, -0.009454, … 0.001481]"
…,…,…,…,…,…,…,…,…,…,…
"""Éomer, King of Rohan""","""f2c11695-f22b-44d5-937c-2578f2…","""{3}{R}{W}""","""Legendary Creature — Human Nob…","""Double strike\n~ enters with a…","""2""","""2""",null,"[""rare""]","[""LTC""]","[0.014415, 0.012861, … 0.019452]"
"""Éomer, Marshal of Rohan""","""fba68512-f536-4961-9e24-563270…","""{2}{R}{R}""","""Legendary Creature — Human Kni…","""Haste\nWhenever one or more ot…","""4""","""4""",null,"[""rare""]","[""PLTR"", ""LTR""]","[-0.022492, 0.017429, … 0.046833]"
"""Éowyn, Fearless Knight""","""c1b37891-5ed9-47e4-8d2f-c2bfd8…","""{2}{R}{W}""","""Legendary Creature — Human Kni…","""Haste\nWhen ~ enters, exile ta…","""3""","""4""",null,"[""rare""]","[""PLTR"", ""LTR""]","[-0.033708, 0.007089, … 0.02833]"


In [4]:
embeds = df["embedding"].to_numpy()
embeds.shape

(32254, 768)

Workaround CUML UMAP bug: https://github.com/rapidsai/cuml/issues/5707#issuecomment-2618102235

In [5]:
from cuml.manifold.umap import fuzzy_simplicial_set, simplicial_set_embedding
from umap.spectral import spectral_layout
from cuml.manifold.umap_utils import find_ab_params

class UMAP:
    def __init__(self, n_epochs, n_neighbors, n_components, min_dist=0.1, random_state=np.random.randint(1, 1000), verbose=False):
        self.n_epochs = n_epochs
        self.n_neighbors = n_neighbors
        self.n_components = n_components
        self.min_dist = min_dist
        self.random_state = random_state
        self.verbose = verbose


        
    def fit_transform(self, X):
        graph = fuzzy_simplicial_set(X, 
                                     n_neighbors=self.n_neighbors, 
                                     random_state=self.random_state, 
                                     metric='euclidean', 
                                     verbose=self.verbose)
        
        layout = spectral_layout(X, graph.tocsr().get(), 
                                 dim=self.n_components, 
                                 random_state=self.random_state)
        spread = 1.0
        a, b = find_ab_params(spread, self.min_dist)
        embedding = simplicial_set_embedding(X, graph, 
                                             init=layout,
                                             a=a, b=b,
                                             n_epochs=self.n_epochs,
                                             n_components=self.n_components,
                                             random_state=self.random_state, 
                                             verbose=self.verbose)
        
        return embedding


In [6]:
%%time
output_dims = 2

# umap_fitted = cuml.UMAP(random_state=None,
#                         init="random",
#                         n_components=output_dims,
#                         n_neighbors=2,
#                         min_dist=0.1,
#                         build_algo="nn_descent",
#                         n_epochs=500_000)

umap_fitted = UMAP(n_components=output_dims,
                        n_neighbors=30,
                        min_dist=0.0,
                        n_epochs=1_000_000)

embeds_t = umap_fitted.fit_transform(embeds)
# _ = umap_fitted.fit(embeds)

CPU times: user 3min 31s, sys: 2.43 s, total: 3min 33s
Wall time: 3min 28s


In [7]:
embeds_np = embeds_t.to_output().get()
embeds_np.shape

(32254, 2)

In [8]:
print(embeds_np[:, 0].min(), embeds_np[:, 0].max())
print(embeds_np[:, 1].min(), embeds_np[:, 1].max())

311.86987 328.27594
-1075.5342 -1061.0306


In [9]:
x_centered = embeds_np[:, 0] - embeds_np[:, 0].mean()
y_centered = embeds_np[:, 1] - embeds_np[:, 1].mean()

print(x_centered.min(), x_centered.max())
print(y_centered.min(), y_centered.max())

-12.268768 4.1372986
-5.8085938 8.694946


In [10]:
df_embeds = df.with_columns(
    x_2d=x_centered,
    y_2d=y_centered,
)

df_embeds

name,scryfallId,manaCost,type,text,power,toughness,loyalty,rarities,sets,embedding,x_2d,y_2d
str,str,str,str,str,str,str,str,list[enum],list[enum],"array[f32, 768]",f32,f32
"""""Ach! Hans, Run!""""","""84f2c8f5-8e11-4639-b7de-00e4a2…","""{2}{R}{R}{G}{G}""","""Enchantment""","""At the beginning of your upkee…",null,null,null,"[""rare""]","[""UNH""]","[0.021458, -0.036102, … -0.00607]",-0.261871,2.217163
"""""Brims"" Barone, Midway Mobster""","""68832214-2943-4253-8884-ffa490…","""{3}{W}{B}""","""Legendary Creature — Human Rog…","""When ~ enters, put a +1/+1 cou…","""5""","""4""",null,"[""uncommon""]","[""UNF""]","[0.000261, 0.025207, … 0.005641]",0.045746,-0.990356
"""""Lifetime"" Pass Holder""","""42293306-aaea-4542-8df4-813823…","""{B}""","""Creature — Zombie Guest""","""This creature enters tapped.\n…","""2""","""1""",null,"[""rare""]","[""UNF""]","[-0.004467, -0.016707, … 0.001401]",-1.034943,0.165527
"""""Name Sticker"" Goblin""","""fd1442b4-da59-4042-835f-143c8d…","""{2}{R}""","""Creature — Goblin Guest""","""When this creature enters from…","""2""","""2""",null,"[""common""]","[""UNF""]","[-0.03243, -0.006905, … -0.026888]",-3.05658,-2.964111
"""""Rumors of My Death . . .""""","""cb3587b9-e727-4f37-b4d6-1baa73…","""{2}{B}""","""Enchantment""","""{3}{B}, Exile a permanent you …",null,null,null,"[""uncommon""]","[""UST""]","[-0.008719, -0.009454, … 0.001481]",0.421021,2.189697
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Éomer, King of Rohan""","""f2c11695-f22b-44d5-937c-2578f2…","""{3}{R}{W}""","""Legendary Creature — Human Nob…","""Double strike\n~ enters with a…","""2""","""2""",null,"[""rare""]","[""LTC""]","[0.014415, 0.012861, … 0.019452]",-1.087738,-2.871826
"""Éomer, Marshal of Rohan""","""fba68512-f536-4961-9e24-563270…","""{2}{R}{R}""","""Legendary Creature — Human Kni…","""Haste\nWhenever one or more ot…","""4""","""4""",null,"[""rare""]","[""PLTR"", ""LTR""]","[-0.022492, 0.017429, … 0.046833]",-1.120117,-2.854126
"""Éowyn, Fearless Knight""","""c1b37891-5ed9-47e4-8d2f-c2bfd8…","""{2}{R}{W}""","""Legendary Creature — Human Kni…","""Haste\nWhen ~ enters, exile ta…","""3""","""4""",null,"[""rare""]","[""PLTR"", ""LTR""]","[-0.033708, 0.007089, … 0.02833]",-0.91925,-3.448242


In [11]:
df_embeds.write_parquet("mtg_embeddings_2d.parquet")